In [64]:
from __future__ import print_function

import pandas as pd
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import plotly
from plotly.offline import iplot
import plotly.express as px
import plotly.graph_objects as go
import folium

In [65]:
data = pd.read_csv("Nigeria covid-19 case.csv")

confirmed = data["Total_Confirmed_case"]
death = data["Total_Death"]
active = data["Total_Active_case"]
state = data["State"]
discharged = data["Total_Recovered"]

In [66]:
state_coordinates = pd.read_csv("MyWebApp/Nigeria State Coordinates.csv")

st = list(state_coordinates["STATES"])
lat = state_coordinates["LAT"]
long = state_coordinates["LONG"]

## INTERACTIVE COVID-19 DASHBOARD MONITORING NIGERIA CASES

The COVID-19 pandemic in Nigeria is part of the worldwide pandemic of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The first confirmed case in Nigeria was announced on 27 of February, 2020, when an Italian citizen in Lagos tested positive for the virus. The Italian citizen who works in Nigeria had returned on 25 of February from Milan, Italy through the Murtala Muhammed International Airport, fell ill on 26 February and was transferred to Lagos State biosecurity facilities for isolation and testing.
On 9 March 2020, a second case of the virus was reported in Ewekoro, Ogun State, a Nigerian citizen who had contact with the Italian citizen

In [67]:
webMap = folium.Map(location=[9.08,7.53], tiles="CartoDB positron", zoom_start=6, max_zoom=6, min_zoom=6)
confirm = np.array(confirmed)
num_st = len(st)
num_state = len(state)

def size(i):
    global sx
    for j in range (len(state)):
        state2 = str(state[j]).upper()
        if st[i] == state2:
            value = confirm[j]
            ac = active[j]
            dth = death[j]
            dc = discharged[j]
            if value > 1000:
                sx = 25
            elif 1000 > value > 500:
                sx = 15
            elif 500 > value > 200:
                sx = 10
            elif 200 > value > 1:
                sx = 5
            else:
                sx = 0
        else:
            pass
    return sx

def val(i,d):
    global d_out
    for j in range (len(state)):
        state2 = str(state[j]).upper()
        if st[i] == state2:
            if d == 'cc':
                d_out = confirm[j]
            if d == ac:
                d_out = active[j]
            if d == dth:
                d_out = death[j]
            if d == dc:
                d_out = discharged[j]
        else:
            pass
    return d_out



for i in range (num_st):
    folium.CircleMarker(
        location=[lat[i],long[i]], 
        fill = True,
        radius = size(i),
        fill_color = "red",
        color = "red",
        tooltip = "%s<br><br>Active: %s<br>Recovered: %s<br>Deaths: %s,"%(st[i],val(i,ac),val(i,dc),val(i,dth))
        ).add_to(webMap)

    
webMap

Filtering through the 36 states to find the top 10 states in each category

In [68]:
def query(select):
    if select == "confirmed":
        fig = px.scatter(data.head(10), x="State", y="Total_Confirmed_case", size="Total_Confirmed_case", 
                         color="State", hover_name="State", size_max=60, title="10 States with the Highest Confirmed Cases")
        fig.update_layout()
    elif select == "active":
        fig = px.scatter(data.head(10).sort_values("Total_Active_case", ascending=False), 
                         x="State", y="Total_Active_case", size="Total_Active_case", color="State", hover_name="State", size_max=50, title="10 States with the Highest Active Cases")
        fig.update_layout()
    elif select == "discharged":
        fig = px.scatter(data.head(10).sort_values("Total_Recovered", ascending=False), 
                         x="State", y="Total_Recovered", size="Total_Recovered", color="State", hover_name="State", size_max=40, title="10 States with the Highest Recovery")
        fig.update_layout()
    elif select == "death":
        fig = px.scatter(data.head(10).sort_values("Total_Death", ascending=False), 
                         x="State", y="Total_Death", size="Total_Death", color="State", hover_name="State", size_max=30, title="10 States with the Highest Number of Deaths")
        fig.update_layout()
    fig.show()
    
interact(query, select=["confirmed","active","discharged","death"]);

interactive(children=(Dropdown(description='select', options=('confirmed', 'active', 'discharged', 'death'), v…

In [69]:
def Search_by_case_type (case):
    if case == "Confirmed Cases":
        trace = go.Bar(x=state, y=confirmed, name='Confirmed Cases')
        layout = go.Layout(title = "Total Confirmed Case in Each State")
    if case == "Active Case":
        trace = go.Bar(x=state, y=active, name='Total Active Case in Each State')
        layout = go.Layout(title = "Active Cases")
    if case == "Discharged":
        trace = go.Bar(x=state, y=discharged, name='Discharged')
        layout = go.Layout(title = "Total Discharged in Each State")
    if case == "Death":
        trace = go.Bar(x=state, y=death, name='Deaths')
        layout = go.Layout(title = "Death Toll")

    figure = go.Figure(data = trace, layout = layout)
    return figure.show()

interact(Search_by_case_type, case=["Confirmed Cases", "Active Case", "Discharged", "Death"]);

interactive(children=(Dropdown(description='case', options=('Confirmed Cases', 'Active Case', 'Discharged', 'D…

In [70]:
total_active = np.sum(np.array(active))
recovered  = np.sum(np.array(discharged))
total_death = np.sum(np.array(death))
total_confirmed = np.sum(np.array(confirmed))


def select_state(Select_State):
    st = Select_State.lower()
    if st == "nigeria":
        ac = total_active
        dc = recovered
        dth = total_death
        print("Total Confirmed: ", total_confirmed)
        report = ['Active Case', 'Death', 'Recovered']
        value = [ac, dth, dc]
        trace = go.Pie(labels = report, values = value)
        data = [trace]
    for i in range(len(state)):
        state1 = str(state[i]).lower()
        if state1 == st:
            print("Total confirmed: ", confirmed[i])
            ac = active[i]
            dc = discharged[i]
            dth = death[i]
            report = ['Active Case', 'Death', 'Recovered']
            value = [ac, dth, dc]
            trace = go.Pie(labels = report, values = value, title = "")
            data = [trace]
        else:
            pass
    fig = go.Figure(data = data)
    iplot(fig)
        
interact(select_state, Select_State=state);

interactive(children=(Dropdown(description='Select_State', options=('Lagos', 'FCT', 'Kano', 'Rivers', 'Oyo', '…

In [49]:
def highlight_col(x):
    r = "background-color: red"
    p = "background-color: purple"
    b = "background-color: blue"
    g = "background-color: green"
    temp_data = pd.DataFrame("", index=x.index, columns = x.columns)
    temp_data.iloc[:,2] = b
    temp_data.iloc[:,3] = g
    temp_data.iloc[:,4] = r
    return temp_data

data.style.apply(highlight_col, axis=None)

,State,Total_Confirmed_case,Total_Active_case,Total_Recovered,Total_Death
0,Lagos,8407,6848,1436,123
1,FCT,1549,1046,475,28
2,Kano,1184,422,712,50
3,Rivers,866,480,356,30
4,Oyo,860,569,282,9
5,Edo,779,587,161,31
6,Ogun,623,250,358,15
7,Kaduna,552,256,286,10
8,Delta,501,360,124,17
9,Borno,466,82,353,31
